In [ ]:
import nltk

###  性别鉴别

In [ ]:
#创建一个分类器的第一步是决定输入的什么样的特征是相关的，以及如何为那些特征编码
 
#以下特征提取器 函数建立一个字典，包含有关给定名称的相关信息:
"""
这个函数返回的字典称为特征集

"""
def gender_features(word):
    return {'last_letter':word[-1]}
gender_features('Shrek')

In [ ]:
#现在，我们已经定义了一个特征提取器，我们需要准备一个例子和对应类标签的链表
from nltk.corpus import names
import random
names = ([(name,'male') for name in names.words('male.txt')]+[(name,'female') for name in names.words('female.txt')])
random.shuffle(names)  # 随机打乱名字
# names

In [ ]:
#接下来，我们使用特征提取器处理名称数据，并划分特征集的结果链表为一个训练集和一个测试集。
#训练集用于训练一个新的“朴素贝叶斯”分类器。
featuresets = [(gender_features(n),g) for (n,g) in names]
train_set,test_set = featuresets[500:],featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [16]:
#测试一些没有出现在训练数据中的名字
classifier.classify(gender_features('Neo'))

'male'

In [17]:
classifier.classify(gender_features('Trinity'))

'female'

In [18]:
print(nltk.classify.accuracy(classifier,test_set))   # 正确率

0.776


In [19]:
#检查分类器，确定哪些特征对于区分名字的性别是最有效的。
classifier.show_most_informative_features(5)#比率称为似然比，可以用于比较不同特征-结果关系

Most Informative Features
             last_letter = 'a'            female : male   =     33.2 : 1.0
             last_letter = 'k'              male : female =     30.3 : 1.0
             last_letter = 'v'              male : female =     17.5 : 1.0
             last_letter = 'f'              male : female =     16.6 : 1.0
             last_letter = 'p'              male : female =     11.9 : 1.0


In [20]:
from nltk.classify import apply_features
train_set = apply_features(gender_features,names[500:])
test_set = apply_features(gender_features,names[:500])

### 选择正确的特征

In [21]:
#建立一个分类器的很多有趣的工作之一是找出哪些特征可能是相关 的，以及我们如何能够表示它们。
#虽然使用相当简单而明显的特征集往往可以得到像样的性 能，
#但是使用精心构建的基于对当前任务的透彻理解的特征，通常会显著提高收益。
 
 
#一个特征提取器，过拟合性别特征。
#这个特征提取器返回的特征集包括大量指 定的特征，从而导致对于相对较小的名字语料库过拟合。
def gender_features2(name):
    featuresets = {}
    featuresets['firstletter'] = name[0].lower()
    featuresets['lastletter'] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        featuresets['count(%s)'%letter] = name.lower().count(letter)
        featuresets['has(%s)'% letter ] = (letter in name.lower())
    return featuresets

gender_features2('John')

{'count(a)': 0,
 'count(b)': 0,
 'count(c)': 0,
 'count(d)': 0,
 'count(e)': 0,
 'count(f)': 0,
 'count(g)': 0,
 'count(h)': 1,
 'count(i)': 0,
 'count(j)': 1,
 'count(k)': 0,
 'count(l)': 0,
 'count(m)': 0,
 'count(n)': 1,
 'count(o)': 1,
 'count(p)': 0,
 'count(q)': 0,
 'count(r)': 0,
 'count(s)': 0,
 'count(t)': 0,
 'count(u)': 0,
 'count(v)': 0,
 'count(w)': 0,
 'count(x)': 0,
 'count(y)': 0,
 'count(z)': 0,
 'firstletter': 'j',
 'has(a)': False,
 'has(b)': False,
 'has(c)': False,
 'has(d)': False,
 'has(e)': False,
 'has(f)': False,
 'has(g)': False,
 'has(h)': True,
 'has(i)': False,
 'has(j)': True,
 'has(k)': False,
 'has(l)': False,
 'has(m)': False,
 'has(n)': True,
 'has(o)': True,
 'has(p)': False,
 'has(q)': False,
 'has(r)': False,
 'has(s)': False,
 'has(t)': False,
 'has(u)': False,
 'has(v)': False,
 'has(w)': False,
 'has(x)': False,
 'has(y)': False,
 'has(z)': False,
 'lastletter': 'n'}

In [22]:
#如果你提供太多的特征，那么该算法将高度依赖你的训练数据的特性而,一般化到新的例子的效果不会很好。
#这 个问题被称为过拟合，当运作在小训练集上时尤其会有问题。
featuresets = [(gender_features2(n),g) for (n,g) in names]
train_set,test_set = featuresets[500:],featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier,test_set))

0.752


In [23]:
#旦初始特征集被选定，完善特征集的一个非常有成效的方法是错误分析。
#首先，我们选择一个开发集，包含用于创建模型的语料数据。
#然后将这种开发集分为训练集和开发测试集。
train_names = names[500:]
devtest_names = names[500:1500]
test_names = names[:500]

In [24]:
#训练集用于训练模型，开发测试集用于进行错误分析，测试集用于系统的最终评估。
trian_set = [(gender_features(n),g) for (n,g) in train_names]
devtest_set = [(gender_features(n),g) for (n,g) in devtest_names]
test_set = [(gender_features(n),g) for (n,g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier,devtest_set))

0.62


In [25]:
#用于训练有监督分类器的语料数据组织图。语料数据分为两类:开发集和测试集。 开发集通常被进一步分为训练集和开发测试集
errors = []
for (name,tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag,guess,name))

In [26]:
#然后，可以检查个别错误案例，在那里该模型预测了错误的标签，尝试确定什么额外信息将使其能够作出正确的决定
#然后可以 相应的调整特征集。我们已经建立的名字分类器在开发测试语料上产生约 100 个错误
for (tag, guess, name) in sorted(errors):
    print('correct=%-8s guess=%-8s name=%-30s'%(tag, guess, name))
    break

correct=male     guess=female   name=Aamir                         


In [27]:
#调整我们的特征提取器包括两个字母后缀的特征:    
def gender_features(word):
    return {'suffix1': word[-1:], 
            'suffix1': word[-2]}
#使用新的特征提取器重建分类器，我们看到测试数据集上的性能提高了近 3 个百分点
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.62


### 文档分类

In [28]:
#建立分类器，自动给新文档添加适当的类别标签
#首先，我们构造一个标记了相应 类别的文档清单。
#对于这个例子，我们选择电影评论语料库，将每个评论归类为正面或负面。
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

对于文档主题识别，我们可以为每个词定义一个特性表示该文档是否包含这个词。
为了限制分类器需要处理的特征的数目，我们一开始构建一个整个语料库中前 2000 个最频繁词的链表。
然后，定义一个特征提取器，简单地检查这些词是否在一个给定的文档中。

In [29]:
#一个文档分类的特征提取器，其特征表示每个词是否在一个给定的文档中
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

In [30]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)'%word] = (word in document_words)
    return features
print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

{'contains(plot)': True, 'contains(:)': True, 'contains(two)': True, 'contains(teen)': False, 'contains(couples)': False, 'contains(go)': False, 'contains(to)': True, 'contains(a)': True, 'contains(church)': False, 'contains(party)': False, 'contains(,)': True, 'contains(drink)': False, 'contains(and)': True, 'contains(then)': True, 'contains(drive)': False, 'contains(.)': True, 'contains(they)': True, 'contains(get)': True, 'contains(into)': True, 'contains(an)': True, 'contains(accident)': False, 'contains(one)': True, 'contains(of)': True, 'contains(the)': True, 'contains(guys)': False, 'contains(dies)': False, 'contains(but)': True, 'contains(his)': True, 'contains(girlfriend)': True, 'contains(continues)': False, 'contains(see)': False, 'contains(him)': True, 'contains(in)': True, 'contains(her)': False, 'contains(life)': False, 'contains(has)': True, 'contains(nightmares)': False, 'contains(what)': True, "contains(')": True, 'contains(s)': True, 'contains(deal)': False, 'contains

In [ ]:
#训练和测试一个分类器进行文档分类。
featuresets = [(document_features(d), c) for (d, c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
print(classifier.show_most_informative_features(5))

In [ ]:
#练一个分类器来 算出哪个后缀最有信息量。首先，让我们找出最常见的后缀:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes[:5])

In [ ]:
#现在，我们已经定义了我们的特征提取器，可以用它来训练一个新的“决策树”的分类器
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [ ]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
print(classifier.classify(pos_features('cats')))

### 探索上下文语境

In [ ]:

#了采取基于词的上下文的特征，我们必须修改以前为我们的特征提取器定义的模式。
#不是只传递已标注的词，我们将传递整个(未标注的)句子，以及目标词的索引
 
#一个词性分类器，它的特征检测器检查一个词出现的上下文以便决定应该分配 的词性标记。特别的，前面的词被作为一个特征。
 
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = '<START>'
    else:
        features["prev-word"] = sentence[i-1]
    return features
print( pos_features(brown.sents()[0], 8))
tagged_sents = brown.tagged_sents(categories='news')
print(tagged_sents)

In [ ]:
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag) )
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

### 序列分类

In [ ]:
#为了捕捉相关的分类任务之间的依赖关系，我们可以使用联合分类器模型，收集有关输 入，选择适当的标签。
#在词性标注的例子中，各种不同的序列分类器 模型可以被用来为一个 给定的句子中的所有的词共同选择词性标签。
 
 
#一种序列分类器策略，称为连续分类或贪婪序列分类，是为第一个输入找到最有可能的 类标签，
#然后使用这个问题的答案帮助找到下一个输入的最佳的标签。
#这个过程可以不断重 复直到所有的输入都被贴上标签


#使用连续分类器进行词性标注。
def pos_features(sentence, i, history): 
     features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
     if i == 0:
         features["prev-word"] = "<START>"
         features["prev-tag"] = "<START>"
     else:
         features["prev-word"] = sentence[i-1]
         features["prev-tag"] = history[i-1]
     return features

In [ ]:
class ConsecutivePosTagger(nltk.TaggerI):
 
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
  def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [ ]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))